#### Training GCN

Model using the graph dataset extracted from the csv file. TO generate csv file, please run the parsing notebook. 

In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.nn as dglnn
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------
# 1. Load Dataset and Prepare Data
# ------------------------------
df = pd.read_csv("all_circuits_features.csv")
print("\n📌 Dataset Overview:")
print(df.info())

gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Convert 'gate_type' to numerical labels
label_encoder = LabelEncoder()
df["gate_label"] = label_encoder.fit_transform(df["gate_type"])

# Feature columns
feature_columns = [
    "fan_in", "fan_out", "dist_to_output", "is_primary_input", "is_primary_output",
    "is_internal", "is_key_gate", "degree_centrality", "betweenness_centrality",
    "closeness_centrality", "clustering_coefficient", "avg_fan_in_neighbors", "avg_fan_out_neighbors"
]

# Drop NaN values and convert features to float
df = df.dropna(subset=feature_columns)
df[feature_columns] = df[feature_columns].astype(float)

# Normalize numerical features
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# ---------------------------
# 2. Build Node and Edge Lists
# ---------------------------
nodes = df["node"].tolist()
node_to_id = {node: i for i, node in enumerate(nodes)}

# Build edge list based on fan-in relationships (directed: from source to current node)
edges = []
for idx, row in df.iterrows():
    node_id = node_to_id[row["node"]]
    # Select potential driving nodes: those with fan_out > 0
    potential_sources = df[df["fan_out"] > 0]["node"].tolist()
    num_fan_in = int(row["fan_in"])
    if num_fan_in > 0:
        sources = potential_sources[:num_fan_in]
        for src in sources:
            if src in node_to_id:
                edges.append((node_to_id[src], node_id))

print("\n🔍 Extracted", len(edges), "edges.")
if len(edges) == 0:
    raise ValueError("No edges found! Check your fan_in values.")

# Convert edges into tensors
src_nodes, dst_nodes = zip(*edges) if edges else ([], [])
src_tensor = torch.tensor(src_nodes, dtype=torch.int64)
dst_tensor = torch.tensor(dst_nodes, dtype=torch.int64)
valid_edges = (src_tensor >= 0) & (dst_tensor >= 0) & (src_tensor < len(nodes)) & (dst_tensor < len(nodes))
src_tensor = src_tensor[valid_edges]
dst_tensor = dst_tensor[valid_edges]

# ---------------------------
# 3. Create the Original DGL Graph
# ---------------------------
graph = dgl.graph((src_tensor, dst_tensor), num_nodes=len(nodes))
graph = dgl.add_self_loop(graph)  # add self-loops to avoid zero in-degree issues

# Assign node features and labels
graph.ndata['features'] = torch.tensor(df[feature_columns].values, dtype=torch.float32)
graph.ndata['labels'] = torch.tensor(df["gate_label"].values, dtype=torch.long)

# Create train-test split using node indices
nodes_idx = np.arange(len(nodes))
train_idx, test_idx = train_test_split(nodes_idx, test_size=0.2, random_state=42)
train_mask = torch.tensor(train_idx, dtype=torch.long)
test_mask = torch.tensor(test_idx, dtype=torch.long)

# ---------------------------
# 4. Define the GCNN Model
# ---------------------------
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, hidden_feats, allow_zero_in_degree=True)
        self.conv2 = dglnn.GraphConv(hidden_feats, out_feats, allow_zero_in_degree=True)

    def forward(self, g, inputs):
        x = self.conv1(g, inputs)
        x = torch.relu(x)
        x = self.conv2(g, x)
        return x

in_feats = len(feature_columns)
hidden_feats = 32
out_feats = len(label_encoder.classes_)
model = GCN(in_feats, hidden_feats, out_feats)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# ---------------------------
# 5. Training Loop
# ---------------------------
epochs = 50
for epoch in range(epochs):
    model.train()
    logits = model(graph, graph.ndata['features'])
    loss = loss_fn(logits[train_mask], graph.ndata['labels'][train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# ---------------------------
# 6. Evaluation
# ---------------------------
model.eval()
with torch.no_grad():
    logits = model(graph, graph.ndata['features'])
    test_logits = logits[test_mask]
    test_predictions = test_logits.argmax(dim=1)
    orig_accuracy = (test_predictions == graph.ndata['labels'][test_mask]).float().mean().item()
    
    print(f"\n✅ Test Accuracy on Original Graph: {orig_accuracy * 100:.2f}%")
    
    true_labels_orig = graph.ndata['labels'][test_mask].cpu().numpy()
    pred_labels_orig = test_predictions.cpu().numpy()
    conf_mat_orig = confusion_matrix(true_labels_orig, pred_labels_orig)
    
    print("\n📝 Confusion Matrix (Original Graph):")
    print(conf_mat_orig)
    
    print("\nClassification Report (Original Graph):")
    print(classification_report(true_labels_orig, pred_labels_orig, target_names=label_encoder.classes_))
   


📌 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60882 entries, 0 to 60881
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   circuit_name            60882 non-null  object 
 1   node                    60882 non-null  object 
 2   gate_type               60882 non-null  object 
 3   fan_in                  60882 non-null  int64  
 4   fan_out                 60882 non-null  int64  
 5   depth                   60882 non-null  object 
 6   dist_to_output          60882 non-null  int64  
 7   is_primary_input        60882 non-null  int64  
 8   is_primary_output       60882 non-null  int64  
 9   is_internal             60882 non-null  int64  
 10  is_key_gate             60882 non-null  int64  
 11  key_dependency          122 non-null    object 
 12  degree_centrality       60882 non-null  float64
 13  betweenness_centrality  60882 non-null  float64
 14  closeness_central

C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Jacobian 
Calculation for one sample.

In [2]:
#########################
# 7. Jacobian Computation and Verification
#########################

print("\n--- Jacobian Computation and Verification ---\n")

# Select one test node. Here, we simply choose the first node in the test set.
test_idx = test_mask[0].item()
print(f"Selected test node index: {test_idx}")

# Get the feature vector for this test node and set it to require gradients.
x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
print("Original feature vector for test node (x0):", x0)

# Define a helper function that takes a feature vector for test_idx and returns the model's output (logit vector)
def f(x):
    """
    Given an input feature vector x for the selected test node (test_idx),
    create a new features tensor by replacing the test node's features and compute the model's output.
    """
    # Clone the original features from the graph.
    new_features = graph.ndata['features'].clone().detach()
    # Replace the test node's feature vector with x.
    new_features[test_idx] = x
    # Forward propagate through the model (the graph structure remains unchanged).
    output = model(graph, new_features)
    # Return the output (logit vector) for the test node.
    return output[test_idx]

# Compute the Jacobian of f with respect to x0.
# The resulting Jacobian has shape (num_classes, feature_dim)
jacobian = torch.autograd.functional.jacobian(f, x0)

print("\nJacobian shape:", jacobian.shape)
print("Jacobian matrix:\n", jacobian)

#########################
# 8. Jacobian Verification via Finite Differences
#########################

# Choose a small random perturbation delta.
epsilon = 1e-3
delta = epsilon * torch.randn_like(x0)
print("\nPerturbation delta vector:", delta)

# Compute the predicted change in output using the linear approximation: J * delta.
predicted_change = jacobian.mv(delta)  # matrix-vector product
print("\nPredicted change in output (Jacobian * delta):", predicted_change)

# Now compute the actual change in output by explicitly perturbing x0.
f_x0 = f(x0)
f_x0_perturbed = f(x0 + delta)
actual_change = f_x0_perturbed - f_x0
print("Actual change in output after perturbation:", actual_change)

# Calculate the relative error between the predicted change and the actual change.
relative_error = (predicted_change - actual_change).norm() / (actual_change.norm() + 1e-8)
print("\nRelative error between predicted and actual change: {:.4e}".format(relative_error.item()))

#########################
# 9. Outcome Interpretation
#########################

if relative_error < 1e-2:
    print("\nOutcome: The Jacobian accurately captures the local sensitivity of the model's output relative to small perturbations in the input.")
else:
    print("\nOutcome: The relatively high error indicates that higher-order effects or estimation noise may be present, suggesting that the local model behavior is less linear.")

#############################################
# 10. Discussion: Verifying Jacobian Information
#############################################
#
# The procedure above gives us two key pieces of information:
#
# 1. The Jacobian matrix tells us how significantly each output logit is affected by changes in each feature component.
#    - For example, large entries in the Jacobian indicate that a small change in that particular input dimension produces a large change in the output.
#
# 2. By comparing the linear prediction (via the Jacobian) to the actual output change for a small input alteration,
#    we verify (via low relative error) that the Jacobian is a valid local approximation.
#
# In the context of circuit analysis:
# - High local sensitivity (i.e. large norms of the Jacobian) might indicate that the GCNN's decision function is "sharp" or brittle in that region.
# - An adversary could use such information to target the most sensitive input directions, while a defender might regularize the model to minimize the Jacobian norm.
#
# Verification:
# - The relative error is used as a measure: If it is low (e.g., < 1%), then the linear approximation (Jacobian) is a reliable indicator of local sensitivity.
# - This provides confidence that the model's predictions are indeed highly sensitive to specific segments of the input space.



--- Jacobian Computation and Verification ---

Selected test node index: 54063
Original feature vector for test node (x0): tensor([ 0.5852,  3.3096, -0.2307, -0.2018, -0.1490,  0.2546, -0.0448,  0.2462,
         0.4904, -0.4170, -0.2078,  0.2185,  0.1667], requires_grad=True)

Jacobian shape: torch.Size([8, 13])
Jacobian matrix:
 tensor([[ 1.1751,  0.1264,  0.3150,  0.0690, -0.1401, -0.2460, -0.9988, -0.4929,
         -0.4157, -1.2658, -0.9800,  0.3919, -1.0841],
        [-0.3199, -0.1870,  0.1485,  1.1556,  0.6583, -1.2490,  0.1671,  0.7602,
         -0.3451,  0.0817,  0.0779,  0.4733, -0.0856],
        [ 0.1512, -0.4985, -1.3084, -0.3012, -0.1043,  0.5629, -0.2641,  0.9349,
         -0.4111,  0.1425,  0.4399,  0.8889, -0.6913],
        [ 0.2978, -0.1672, -0.3140,  0.2385, -0.0732,  0.3313, -0.1602, -0.1774,
          0.4853,  0.2406,  1.1527, -0.2559,  0.2984],
        [-2.5618, -0.0189,  0.2247, -0.6293,  0.4530,  1.2460, -0.2286, -0.2451,
         -0.2844, -0.6528, -0.1483,  0.529

##### Several Samples per class

In [5]:
import torch
import numpy as np

# Define gate types corresponding to the labels
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store metrics for each class
class_metrics = {gt: {'jacobian_norms': [], 'relative_errors': []} for gt in gate_types}
overall_jacobian_norms = []
overall_relative_errors = []

# Function to compute the Jacobian for a given test index.
def compute_jacobian_for_sample(test_idx):
    # Get the feature vector for the selected test node and require gradients.
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    def f(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        output = model(graph, new_features)
        return output[test_idx]
    # Compute the Jacobian of f with respect to x0.
    jacobian = torch.autograd.functional.jacobian(f, x0)
    return jacobian, x0, f

# Loop over each selected test node (100 per class)
for test_idx in selected_test_nodes:
    # Determine the ground-truth label and corresponding gate type.
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    # Compute the Jacobian for the selected test node.
    jacobian, x0, f = compute_jacobian_for_sample(test_idx)
    # Compute the Frobenius norm of the Jacobian.
    jacobian_norm = torch.norm(jacobian, p='fro').item()
    
    # Finite-difference verification:
    epsilon = 1e-3
    delta = epsilon * torch.randn_like(x0)
    predicted_change = jacobian.mv(delta)
    f_x0 = f(x0)
    f_x0_perturbed = f(x0 + delta)
    actual_change = f_x0_perturbed - f_x0
    relative_error = (torch.norm(predicted_change - actual_change) /
                      (torch.norm(actual_change) + 1e-8)).item()
    
    # Store values for the current sample.
    class_metrics[class_name]['jacobian_norms'].append(jacobian_norm)
    class_metrics[class_name]['relative_errors'].append(relative_error)
    overall_jacobian_norms.append(jacobian_norm)
    overall_relative_errors.append(relative_error)

# Display per-class results in a tabular format.
print("\nClass-wise Jacobian Analysis Results:")
header = "{:<10s} {:>30s} {:>35s}".format("Class", "Avg. Jacobian Norm ± Std", "Avg. Relative Error ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    norms = class_metrics[cls]['jacobian_norms']
    errors = class_metrics[cls]['relative_errors']
    avg_norm = np.mean(norms) if norms else 0
    std_norm = np.std(norms) if norms else 0
    avg_rel_error = np.mean(errors) if errors else 0
    std_rel_error = np.std(errors) if errors else 0
    print("{:<10s} {:>15.4f} ± {:<12.4f} {:>15.4e} ± {:<10.4e}".format(
        cls, avg_norm, std_norm, avg_rel_error, std_rel_error))

# Compute overall aggregated metrics.
overall_avg_norm = np.mean(overall_jacobian_norms)
overall_std_norm = np.std(overall_jacobian_norms)
overall_avg_rel_error = np.mean(overall_relative_errors)
overall_std_rel_error = np.std(overall_relative_errors)

print("\nOverall Aggregated Jacobian Analysis Results:")
print("Average Jacobian Norm: {:.4f} ± {:.4f}".format(overall_avg_norm, overall_std_norm))
print("Average Relative Error: {:.4e} ± {:.4e}".format(overall_avg_rel_error, overall_std_rel_error))



Class-wise Jacobian Analysis Results:
Class            Avg. Jacobian Norm ± Std           Avg. Relative Error ± Std
-----------------------------------------------------------------------------
and                 7.8267 ± 1.8320            1.6930e-03 ± 1.5638e-02
or                  3.7826 ± 0.1303            3.0280e-04 ± 1.4921e-04
nand                6.4220 ± 2.1018            3.0992e-03 ± 2.9427e-02
nor                 4.7018 ± 1.1635            1.8139e-04 ± 1.3186e-04
xor                 8.0677 ± 1.0923            1.1946e-03 ± 6.7491e-03
xnor                6.4301 ± 2.6591            5.6770e-03 ± 5.3800e-02
buf                 3.4224 ± 0.3137            2.2229e-03 ± 1.3015e-02
not                 3.7574 ± 0.2653            5.2594e-04 ± 3.4820e-04

Overall Aggregated Jacobian Analysis Results:
Average Jacobian Norm: 5.7233 ± 2.3320
Average Relative Error: 1.9902e-03 ± 2.4107e-02


#### Local Lipschitz constant

In [6]:
import torch
import numpy as np

print("\n--- Local Lipschitz Constant Computation and Verification (100 Samples Per Class) ---\n")

# Initialize dictionaries to store the local Lipschitz constants (L_i) for each class.
class_lipschitz = {gt: [] for gt in gate_types}
overall_lipschitz = []

def compute_jacobian_for_sample(test_idx):
    """
    Compute the Jacobian matrix for a given test index.
    Returns the Jacobian, the original input x0, and the output function f.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    
    def f(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        output = model(graph, new_features)
        return output[test_idx]
    
    jacobian = torch.autograd.functional.jacobian(f, x0)
    return jacobian, x0, f

# Iterate over each selected test node from the chosen 100 samples per class.
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    # Compute the Jacobian for this sample.
    jacobian, x0, f = compute_jacobian_for_sample(test_idx)
    
    # The local Lipschitz constant is approximated as the spectral norm (i.e., the largest singular value)
    # of the Jacobian matrix.
    L_local = torch.linalg.norm(jacobian, ord=2).item()
    
    # Store the value per class and overall.
    class_lipschitz[class_name].append(L_local)
    overall_lipschitz.append(L_local)

# Print the per-class results in a tabular format.
print("\nClass-wise Local Lipschitz Constant Results:")
header = "{:<10s} {:>30s}".format("Class", "Avg Lipschitz Constant ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    values = class_lipschitz[cls]
    if len(values) > 0:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val = 0
        std_val = 0
    print("{:<10s} {:>15.4f} ± {:<12.4f}".format(cls, avg_val, std_val))
    
# Compute and display the overall aggregated metrics.
overall_avg = np.mean(overall_lipschitz)
overall_std = np.std(overall_lipschitz)
print("\nOverall Aggregated Local Lipschitz Constant:")
print("Average Lipschitz Constant: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Local Lipschitz Constant Computation and Verification (100 Samples Per Class) ---


Class-wise Local Lipschitz Constant Results:
Class        Avg Lipschitz Constant ± Std
-----------------------------------------
and                 5.8381 ± 1.3819      
or                  2.6307 ± 0.0743      
nand                4.7173 ± 1.5420      
nor                 3.2976 ± 0.9815      
xor                 6.7301 ± 1.0445      
xnor                4.6441 ± 1.8426      
buf                 2.3263 ± 0.1241      
not                 2.7390 ± 0.2534      

Overall Aggregated Local Lipschitz Constant:
Average Lipschitz Constant: 4.2474 ± 1.9138


#### Hessian-Based Curvature Measure

In [11]:
import torch
import numpy as np

print("\n--- Hessian-Based Curvature Measure Computation (100 Samples Per Class) ---\n")

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store the maximum eigenvalue (lambda_max) for each gate type.
class_hessian_eig = {gt: [] for gt in gate_types}
overall_hessian_eig = []

def compute_hessian_for_sample(test_idx):
    """
    Compute the Hessian-based curvature measure for a given test sample.
    We define a scalar function h(x) as the log probability (via log-softmax) 
    of the predicted class (determined on the unperturbed input). This helps
    introduce nonlinearity, thereby yielding nonzero second-order derivatives.
    """
    # Retrieve the feature vector for the test node and require gradients.
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    
    # Determine the predicted class for the test node.
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])
        pred_class = torch.argmax(logits[test_idx])
    
    # Define a scalar function h(x) that returns log probability for the predicted class.
    def h(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        logits = model(graph, new_features)[test_idx]
        log_probs = torch.log_softmax(logits, dim=0)
        return log_probs[pred_class]
    
    # Compute the Hessian of h(x) at x0.
    H = torch.autograd.functional.hessian(h, x0, create_graph=False, vectorize=False)
    H = H.detach()
    
    # Compute eigenvalues using a symmetric eigendecomposition.
    eigvals = torch.linalg.eigvalsh(H)
    lambda_max = torch.max(eigvals).item()
    return lambda_max, x0, h

# Iterate over each selected test node (assumed to be 100 samples per class).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    lambda_max, x0, h_func = compute_hessian_for_sample(test_idx)
    class_hessian_eig[class_name].append(lambda_max)
    overall_hessian_eig.append(lambda_max)

# Display per-class results in a tabular format.
print("\nClass-wise Hessian-Based Curvature Results:")
header = "{:<10s} {:>35s}".format("Class", "Avg. Max Eigenvalue ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    values = class_hessian_eig[cls]
    if len(values) > 0:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))

# Compute and display the overall aggregated result.
overall_avg = np.mean(overall_hessian_eig)
overall_std = np.std(overall_hessian_eig)
print("\nOverall Aggregated Hessian-Based Curvature Measure:")
print("Average Max Eigenvalue: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Hessian-Based Curvature Measure Computation (100 Samples Per Class) ---


Class-wise Hessian-Based Curvature Results:
Class                Avg. Max Eigenvalue ± Std
----------------------------------------------
and                 0.0000 ± 0.0000         
or                  0.0000 ± 0.0000         
nand                0.0000 ± 0.0000         
nor                 0.0000 ± 0.0000         
xor                 0.0000 ± 0.0000         
xnor                0.0000 ± 0.0000         
buf                 0.0000 ± 0.0000         
not                 0.0000 ± 0.0000         

Overall Aggregated Hessian-Based Curvature Measure:
Average Max Eigenvalue: 0.0000 ± 0.0000


#### Prediction Margin

In [12]:
import torch
import numpy as np

print("\n--- Prediction Margin Computation (100 Samples Per Class) ---\n")

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store prediction margins for each gate type.
class_margin = {gt: [] for gt in gate_types}
overall_margin = []

# Iterate over each selected test node.
for test_idx in selected_test_nodes:
    # Obtain model output (logit vector) for the test sample.
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])[test_idx]  # f(x_i)
    
    # Predicted class: argmax_j f_j(x_i)
    pred_class = int(torch.argmax(logits))
    pred_logit = logits[pred_class].item()
    
    # Obtain the second highest value: set pred_class index to a very low number,
    # then take the max of the rest.
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()
    
    # Prediction margin: f_{y_pred}(x_i) - max_{j \neq y_pred} f_j(x_i)
    margin = pred_logit - second_max
    
    # Group the margin by the actual gate type label.
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    class_margin[class_name].append(margin)
    overall_margin.append(margin)

# Display per-class results in a tabular format.
print("\nClass-wise Prediction Margin Results:")
header = "{:<10s} {:>40s}".format("Class", "Avg. Margin ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_margin[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))

# Compute and display the overall aggregated prediction margin.
overall_avg_margin = np.mean(overall_margin)
overall_std_margin = np.std(overall_margin)
print("\nOverall Aggregated Prediction Margin:")
print("Average Margin: {:.4f} ± {:.4f}".format(overall_avg_margin, overall_std_margin))



--- Prediction Margin Computation (100 Samples Per Class) ---


Class-wise Prediction Margin Results:
Class                             Avg. Margin ± Std
---------------------------------------------------
and                 3.4587 ± 1.4896         
or                  5.7445 ± 0.8630         
nand                1.2225 ± 0.6649         
nor                 1.4505 ± 0.7046         
xor                 4.8346 ± 1.0507         
xnor                2.0727 ± 1.1462         
buf                 6.7341 ± 0.5587         
not                 4.5673 ± 0.4763         

Overall Aggregated Prediction Margin:
Average Margin: 3.6833 ± 2.2053


#### Adversarial Robustness Radius

In [13]:
import torch
import numpy as np

print("\n--- Adversarial Robustness Radius Computation (100 Samples Per Class) ---\n")

# Dictionary to store the adversarial robustness radius for each gate type.
class_adv_radius = {gt: [] for gt in gate_types}
overall_adv_radius = []

def f_for_sample(x, test_idx):
    """
    Given an input x (modified feature vector for the test node),
    returns the output (logit vector) for that node.
    """
    new_features = graph.ndata['features'].clone().detach()
    new_features[test_idx] = x
    with torch.no_grad():
        out = model(graph, new_features)
    return out[test_idx]

def adversarial_radius_for_sample(test_idx, initial_epsilon=1e-3, growth_factor=1.2, max_epsilon=10.0, bs_iters=10, num_trials=10):
    """
    For a given test node (identified by test_idx), this function computes the smallest
    perturbation norm (adversarial robustness radius) required to change the model's prediction.
    We sample a number of random directions and, for each, increase the perturbation until
    the predicted class changes; a binary search is then performed to refine the minimal epsilon.
    The minimal epsilon over all trials is returned.
    """
    # Retrieve the original feature vector.
    x0 = graph.ndata['features'][test_idx].clone().detach()
    
    # Determine the predicted class at x0.
    with torch.no_grad():
        out = model(graph, graph.ndata['features'])
        y0 = torch.argmax(out[test_idx]).item()
        
    # Helper function: returns True if prediction at x remains unchanged.
    def is_same(x):
        out = f_for_sample(x, test_idx)
        return (torch.argmax(out).item() == y0)
    
    radii = []
    for _ in range(num_trials):
        # Sample a random direction.
        d = torch.randn_like(x0)
        d = d / (torch.norm(d) + 1e-8)
        epsilon = initial_epsilon
        
        # Increase epsilon until the classification changes or max_epsilon is reached.
        while epsilon < max_epsilon and is_same(x0 + epsilon * d):
            epsilon *= growth_factor
        
        # If maximum epsilon is reached, use max_epsilon as candidate.
        if epsilon >= max_epsilon:
            candidate = max_epsilon
        else:
            # Refine the candidate using binary search between [epsilon/growth_factor, epsilon].
            low = epsilon / growth_factor
            high = epsilon
            for _ in range(bs_iters):
                mid = (low + high) / 2
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = high
        radii.append(candidate)
    
    return min(radii)

# Iterate over each selected test node (assumed 100 samples per class are in selected_test_nodes).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    radius = adversarial_radius_for_sample(test_idx)
    class_adv_radius[class_name].append(radius)
    overall_adv_radius.append(radius)

# Display per-class results.
print("\nClass-wise Adversarial Robustness Radius Results:")
header = "{:<10s} {:>35s}".format("Class", "Avg. Radius ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_adv_radius[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))
    
# Compute overall aggregated results.
overall_avg = np.mean(overall_adv_radius)
overall_std = np.std(overall_adv_radius)
print("\nOverall Aggregated Adversarial Robustness Radius:")
print("Average Radius: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Adversarial Robustness Radius Computation (100 Samples Per Class) ---



KeyboardInterrupt: 

#### Stability Under Input Noise

In [14]:
import torch
import numpy as np

print("\n--- Stability Under Input Noise Computation (100 Samples Per Class) ---\n")

# Standard deviation for Gaussian noise perturbations.
sigma = 0.01

# Number of noise samples to approximate the expectation.
num_noise_samples = 20

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionaries to store stability metric per gate type and overall.
class_stability = {gt: [] for gt in gate_types}
overall_stability = []

def stability_for_sample(test_idx, sigma, num_samples):
    """
    Compute the stability under input noise for a given test sample.
    For the sample at test_idx, the prediction difference is computed as the Euclidean norm 
    between the model outputs for the original and the noisy inputs. The stability measure 
    S(x_i) is obtained by averaging over multiple noise samples.
    """
    # Retrieve the original feature vector.
    x0 = graph.ndata['features'][test_idx].clone().detach()
    
    # Compute the model output for the unperturbed input.
    with torch.no_grad():
        f_orig = model(graph, graph.ndata['features'])[test_idx]
    
    differences = []
    for _ in range(num_samples):
        # Sample noise from a Gaussian distribution: N(0, sigma^2 I)
        noise = sigma * torch.randn_like(x0)
        x_noisy = x0 + noise
        
        # Replace the test node's feature with the noisy feature.
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x_noisy
        
        # Compute the model's output for the noisy input.
        with torch.no_grad():
            f_noisy = model(graph, new_features)[test_idx]
        
        # Calculate the Euclidean norm difference between the noisy and original outputs.
        diff = torch.norm(f_noisy - f_orig).item()
        differences.append(diff)
    
    # Average difference approximates the expectation.
    return np.mean(differences)

# Iterate over each selected test node (assumed to be 100 samples per class in selected_test_nodes).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    stability_measure = stability_for_sample(test_idx, sigma, num_noise_samples)
    class_stability[class_name].append(stability_measure)
    overall_stability.append(stability_measure)

# Display per-class results.
print("\nClass-wise Stability Under Input Noise Results:")
header = "{:<10s} {:>40s}".format("Class", "Avg. Stability (S(x)) ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_stability[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))
    
overall_avg = np.mean(overall_stability)
overall_std = np.std(overall_stability)
print("\nOverall Aggregated Stability Under Input Noise:")
print("Average Stability: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Stability Under Input Noise Computation (100 Samples Per Class) ---


Class-wise Stability Under Input Noise Results:
Class                   Avg. Stability (S(x)) ± Std
---------------------------------------------------
and                 0.0692 ± 0.0177         
or                  0.0458 ± 0.0049         
nand                0.0597 ± 0.0185         
nor                 0.0458 ± 0.0103         
xor                 0.0810 ± 0.0126         
xnor                0.0535 ± 0.0224         
buf                 0.0383 ± 0.0040         
not                 0.0270 ± 0.0030         

Overall Aggregated Stability Under Input Noise:
Average Stability: 0.0550 ± 0.0204
